In [ ]:
!python3 -m spacy download ru_core_news_sm

In [2]:
import spacy
import pandas as pd

from tqdm import tqdm
from spacy.tokens import DocBin

nlp = spacy.load("ru_core_news_sm")

In [3]:
!git clone 'https://github.com/sismetanin/rureviews.git'

df = pd.read_csv('./rureviews/women-clothing-accessories.3-class.balanced.csv', encoding='utf-8', sep='\t')

fatal: целевой путь «rureviews» уже существует и не является пустым каталогом.


In [4]:
df = df.sample(frac=0.5)

In [5]:
df['sentiment'].value_counts()

sentiment
positive    15008
neautral    14997
negative    14995
Name: count, dtype: int64

In [6]:
df = df[df['sentiment'] != 'neautral'] # q
data = [tuple(df.iloc[i].values) for i in range(df.shape[0])]

In [7]:
# https://github.com/madokaiskami/Nijigen_saikou/blob/cb361aded185cefc7cdae2b87e10a97e02ec5c16/preprocess/datatospacy.py#L16
from sklearn.utils import shuffle
df = shuffle(df)
data = [tuple(df.iloc[i].values) for i in range(df.shape[0])]
flag = int(df.shape[0]*0.8)
train_data = data[:flag]
valid_data = data[flag:]

def make_docs(data):
    docs = []

    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total=len(data)):
        if label == '0':
            doc.cats["positive"] = 1
            doc.cats["anger"] = 0
            doc.cats["disgust"] = 0
            doc.cats["down"] = 0
        elif label == '1':
            doc.cats["positive"] = 0
            doc.cats["anger"] = 1
            doc.cats["disgust"] = 0
            doc.cats["down"] = 0
        elif label == '2':
            doc.cats["positive"] = 0
            doc.cats["anger"] = 0
            doc.cats["disgust"] = 1
            doc.cats["down"] = 0
        elif label == '3':
            doc.cats["positive"] = 0
            doc.cats["anger"] = 0
            doc.cats["disgust"] = 0
            doc.cats["down"] = 1

        docs.append(doc)
    return docs

In [9]:
# we are so far only interested in the first 5000 reviews
# this will keep the training time short. In practice take as much data as you can get.
# you can always reduce it to make the script even faster.
num_texts = 5000

# first we need to transform all the training data
train_docs = make_docs(train_data[:num_texts])

# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")

# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

100%|██████████| 5000/5000 [00:28<00:00, 178.34it/s]


На этом месте мы идем в https://spacy.io/usage/training#quickstart, там настраиваем под себя конфиг (textcat), копируем его руками(!) в базе config.cfg, указываем правильные пути до трейн и вэлид

In [11]:
!python3 -m spacy train config.cfg --output ./output

Usage: python -m spacy train [OPTIONS] CONFIG_PATH
Try 'python -m spacy train --help' for help.
╭─ Error ──────────────────────────────────────────────────────────────────────╮
│ Invalid value for 'CONFIG_PATH': Path 'config.cfg' does not exist.           │
╰──────────────────────────────────────────────────────────────────────────────╯


In [13]:
import spacy
# load the best model from training
nlp = spacy.load("output/model-best")
text = ""
print("type : \"quit\" to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc. cats)
    if doc. cats['positive'] >.5:
        print(+"the sentiment is positive")
    else:
        print(+"the sentiment is negative")

OSError: [E050] Can't find model 'output/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.